In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import importlib
from sklearn.metrics import confusion_matrix


# Ajouter le chemin absolu du dossier 'config'
sys.path.append('../config')
sys.path.append('../src')

# Importer le module de configuration
import config
# Recharger le module de configuration pour prendre en compte les modifications
importlib.reload(config)
# Utiliser la configuration
cfg = config.cfg

from common_utils import load_normalize_images, load_rgb2mask_labels

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

print(tf.__version__)

def load_compile_model(model_path) :
    trained_model = None
    print(model_path)
    
    # Charger le modèle sans l'optimiseur
    try:
        trained_model = load_model(model_path, compile=False)
        print("Modèle chargé avec succès sans l'optimiseur.")
    except TypeError as e:
        print(f"Erreur lors du chargement du modèle : {e}")
    
    # Re-compiler le modèle avec un optimiseur compatible
    try:
        # Utiliser un optimiseur compatible avec TensorFlow 2.10
        optimizer = Adam()  # Vous pouvez ajuster les paramètres selon vos besoins
        trained_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        print("Modèle compilé avec succès.")
    except Exception as e:
        print(f"Erreur lors de la compilation du modèle : {e}")
        
    return trained_model

              
# Fonction de calcul de la sensibilité et de la précision pour une image
def calculate_image_metrics(predicted, real):
    TP = np.sum(predicted == real)
    FN = np.sum((predicted != real) & (real != 0))
    FP = np.sum((predicted != real) & (predicted != 0))

    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0

    return sensitivity, precision, f1

# Fonction pour créer un masque binaire
def create_mask(label_list, background_value=0):
    """Créer un masque binaire pour les pixels non background."""
    mask_list = [(label != background_value) for label in label_list]
    return mask_list


def displayPred_vs_Reality(trained_model, img_list_norm, label_list, loDisplayImg):
    """imgStart/ImgEnd = Range des images a analyse
       loAllDate = True/False si True, on analyse toute les images sinon analyse du range
    """
    mask_list = create_mask(label_list)
    
    # Initialiser les listes pour stocker les résultats
    tumor_detected_list = []
    tumor_detected_real_list = []
    sensitivities = []
    precisions = []
    f1_scores = []
    all_true_labels = []
    all_pred_labels = []

    imgStart = 0 
    imgEnd = len(img_list_norm)-1

    for noImg in range(imgStart, imgEnd + 1):
        # Prédiction
        prediction = trained_model.predict(np.expand_dims(img_list_norm[noImg], axis=0))
        predicted_labels = np.argmax(prediction, axis=-1)[0]
        
        # Appliquer le masque
        filtered_predicted = predicted_labels[mask_list[noImg]]
        filtered_real = label_list[noImg][mask_list[noImg]]
        
        # Stocker toutes les étiquettes pour la matrice de confusion
        all_true_labels.extend(filtered_real.flatten())
        all_pred_labels.extend(filtered_predicted.flatten())
        
        # Calcul de la sensibilité et de la précision
        sensitivity, precision, f1 = calculate_image_metrics(filtered_predicted, filtered_real)
        sensitivities.append(sensitivity)
        precisions.append(precision)
        f1_scores.append(f1)
        
        print(f"No image : {noImg}")
        
        if loDisplayImg:
            # Afficher l'image des probabilités de tumeur
            plt.figure(figsize=(12, 8))
            plt.subplot(1, 2, 1)
            plt.imshow(predicted_labels, cmap='gray')
            plt.title(f'Prédiction {noImg}')
            
            plt.text(120, 20, f'Sensibilité : {sensitivity:.4f}', fontsize=20, color='green' if sensitivity >= 0.95 else 'red', ha='center')
            plt.text(120, 50, f'Précision : {precision:.4f}', fontsize=20, color='green' if precision >= 0.70 else 'red', ha='center')
  
            plt.subplot(1, 2, 2)
            plt.imshow(label_list[noImg], cmap='gray')
            plt.title(f'Image labelisée {noImg}')
            plt.show()

    # Convertir les listes en tableaux NumPy
    mean_sensitivity = np.mean(sensitivities)
    mean_precision = np.mean(precisions)
    mean_f1 = np.mean(f1_scores)

    # Calcul de la matrice de confusion
    cm = confusion_matrix(all_true_labels, all_pred_labels)
    
    return mean_sensitivity, mean_precision, mean_f1, cm

####################################################################################

#Charger le modèle
#Le modèle a été entraîné sur cloud sageMaker Tensorflow 2.16.
#Le test est effectuée en loca sur Tensorflow 2.10


trained_model = load_compile_model("../model/"+ cfg['MODEL_NAME'])

# trained_model = tf.keras.models.load_model("model/"+ cfg['MODEL_NAME'])

if cfg['VALID_OR_TRAIN_MODE']  == 'VALID' :
    selectedDir = cfg['VALID_DIR']
else :
    selectedDir = cfg['TESTS_DIR']
     
if cfg['NBIMG_TO_CHECK'] == None : #traiter toute les images
    total_images = len([name for name in os.listdir(selectedDir + cfg['OUTPUT_DIR_IMAGE']) if os.path.isfile(os.path.join(selectedDir + cfg['OUTPUT_DIR_IMAGE'], name))])-1
else:
    total_images = cfg['NBIMG_TO_CHECK']
     
#Charger et labellisé mask validation
label_list = load_rgb2mask_labels(selectedDir, 0, total_images)

#Charger et normaliser images validation
img_list_norm = load_normalize_images(selectedDir, 0, total_images)

    
# Prediction sur les images
mean_sensitivity, mean_precision, mean_f1, cm = displayPred_vs_Reality(trained_model, img_list_norm, label_list, cfg['DISPLAY_GRAPH'])

print("Sensibilité moyenne:", mean_sensitivity)
print("Précision moyenne:", mean_precision)
print("F1 Score moyen:", mean_f1)
print("Matrice de Confusion:")
print(cm)

















